In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [3]:
import sampler

In [4]:
p = 'input/hm/test/afa5e8098.tiff'

In [5]:
sampler.image_preprocessing_gdal(p, 'test', 4, block_size=(2048,2048))

(36800, 43780) 3


/home/gsm/.local/lib/python3.7/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
  0%|          | 0/6 [00:00<?, ?it/s]

(36800, 43780) (36800, 43780)(36800, 43780)3 
 3
3(36800, 43780) 
3(36800, 43780)
(36800, 43780)  (36800, 43780)3
3 3

(36800, 43780) 3
test 0 0 (3, 2048, 2048) uint8
test 0 0 (3, 2048, 2048) uint8
test 0 0 (3, 2048, 2048) uint8
test 0 0 (3, 2048, 2048) uint8
test 0 1 (3, 2048, 2048) uint8
test 0 1 (3, 2048, 2048) uint8
test 0 1 (3, 2048, 2048) uint8
test 0 1 (3, 2048, 2048) uint8
test 0 2 (3, 2048, 2048) uint8
test 0 2 (3, 2048, 2048) uint8
test 0 2 (3, 2048, 2048) uint8
test 0 2 (3, 2048, 2048) uint8
test 0 3 (3, 2048, 2048) uint8
test 0 3 (3, 2048, 2048) uint8
test 0 3 (3, 2048, 2048) uint8
test 0 3 (3, 2048, 2048) uint8
test 0 4 (3, 2048, 2048) uint8
test 0 4 (3, 2048, 2048) uint8
test 0 4 (3, 2048, 2048) uint8
test 0 4 (3, 2048, 2048) uint8
test 0 5 (3, 2048, 2048) uint8
test 0 5 (3, 2048, 2048) uint8
test 0 5 (3, 2048, 2048) uint8
test 0 5 (3, 2048, 2048) uint8
test 0 6 (3, 2048, 2048) uint8
test 0 6 (3, 2048, 2048) uint8
test 0 6 (3, 2048, 2048) uint8
test 0 6 (3, 2048, 2048) ui

Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "src/sampler.py", line 224, in mp_func_wrapper
    def mp_func_wrapper(func, args):
  File "src/sampler.py", line 224, in mp_func_wrapper
    def mp_func_wrapper(func, args):
  File "src/sampler.py", line 207, in _rea

ConnectionResetError: [Errno 104] Connection reset by peer